In [8]:
# Install necessary libraries
get_ipython().system('pip install -U transformers datasets accelerate peft trl bitsandbytes langchain chromadb sentence-transformers flash-attn')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
# Log in to Hugging Face
get_ipython().system('huggingface-cli login')


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `mariamattiaa` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

In [9]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

In [4]:
# Define paths and model configurations
base_model = "meta-llama/Llama-3.2-3B-Instruct"

In [5]:
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA Available: True
Device Name: Tesla T4


In [7]:
!pip install BitsAndBytesConfig

ERROR: Could not find a version that satisfies the requirement BitsAndBytesConfig (from versions: none)
ERROR: No matching distribution found for BitsAndBytesConfig


In [11]:
import torch
from transformers import BitsAndBytesConfig

# Specify the torch data type
torch_dtype = torch.float16

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [16]:
# Function to generate prompt with Gardiner codes and meanings
def generate_prompt_with_data(dataframe):
    base_prompt = (
        "You are an expert in translating Gardiner codes into meaningful and accurate English sentences. "
        "Your task involves the following steps: \n"
        "1. For a single Gardiner code, provide its meaning clearly and concisely without unnecessary details.\n"
        "2. For multiple Gardiner codes, explain each code individually and then combine their meanings into a coherent sentence.\n"
        "3. If you encounter an unknown Gardiner code, respond with 'I do not know' and avoid fabricating answers.\n"
        "Guidelines: \n"
        "- Keep responses accurate and professional.\n"
        "- Only use information provided in the query.\n"
        "- Avoid referencing unrelated codes unless explicitly requested.\n"
        "Available Gardiner codes and their meanings: \n"
    )
    codes_meanings = "\n".join(
        f"{row['gardiner_code']}: {row['english_translation']}"
        for _, row in dataframe.iterrows()
    )

    return base_prompt + codes_meanings


In [18]:
# Load data and generate the full prompt
data_path = '/content/cleaned_dataset2.csv'
data = pd.read_csv(data_path, encoding='latin1')
explanation_prompt = generate_prompt_with_data(data)

In [19]:
# Generate text using the model
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.5  # Lower temperature for more deterministic output
)

Device set to use cuda:0


In [20]:
# Single Gardiner code query
query = f"{explanation_prompt}\nWhat does Gardiner code 'A5' mean?"
response = llm_pipeline(query)
print(f"Generated Response for Single Code: {response[0]['generated_text']}")


Generated Response for Single Code: You are an expert in translating Gardiner codes into meaningful and accurate English sentences. Your task involves the following steps: 
1. For a single Gardiner code, provide its meaning clearly and concisely without unnecessary details.
2. For multiple Gardiner codes, explain each code individually and then combine their meanings into a coherent sentence.
3. If you encounter an unknown Gardiner code, respond with 'I do not know' and avoid fabricating answers.
Guidelines: 
- Keep responses accurate and professional.
- Only use information provided in the query.
- Avoid referencing unrelated codes unless explicitly requested.
Available Gardiner codes and their meanings: 
A1: man, names
A2: eat, drink, speak, think.
A3: sit.
A4: adoration, hide
A5: hide
A6: purity, cleanliness
A7: weary, weak
A8: jubilation
A9: load, carry, work
A10: sail
A11: friend
A12: army, soldier
A13: enemy
A14: die, enemy
A14a: die, enemy
A15: fall, overthrow
A16: bow
A17: 

In [21]:
# Multiple Gardiner codes query
query = f"{explanation_prompt}\nWhat do Gardiner codes 'A5', 'A10', and 'A3' mean?"
response = llm_pipeline(query)
print(f"Generated Response for Multiple Codes: {response[0]['generated_text']}")

Generated Response for Multiple Codes: You are an expert in translating Gardiner codes into meaningful and accurate English sentences. Your task involves the following steps: 
1. For a single Gardiner code, provide its meaning clearly and concisely without unnecessary details.
2. For multiple Gardiner codes, explain each code individually and then combine their meanings into a coherent sentence.
3. If you encounter an unknown Gardiner code, respond with 'I do not know' and avoid fabricating answers.
Guidelines: 
- Keep responses accurate and professional.
- Only use information provided in the query.
- Avoid referencing unrelated codes unless explicitly requested.
Available Gardiner codes and their meanings: 
A1: man, names
A2: eat, drink, speak, think.
A3: sit.
A4: adoration, hide
A5: hide
A6: purity, cleanliness
A7: weary, weak
A8: jubilation
A9: load, carry, work
A10: sail
A11: friend
A12: army, soldier
A13: enemy
A14: die, enemy
A14a: die, enemy
A15: fall, overthrow
A16: bow
A1

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluation Metrics
def evaluate_model(predictions, references):
    accuracy = accuracy_score(references, predictions)
    precision = precision_score(references, predictions, average='weighted')
    recall = recall_score(references, predictions, average='weighted')
    f1 = f1_score(references, predictions, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

# Example usage
sample_predictions = ["man", "hide", "sit"]
sample_references = ["man", "hide", "sit"]
evaluate_model(sample_predictions, sample_references)


Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
